In [2]:
import datetime
import pandas as pd
import numpy as np

from pyspark import SparkContext
from pyspark import SQLContext
from pyspark.sql.session import SparkSession


from pyspark.sql import DataFrameStatFunctions as statFunc

from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from pyspark.sql.functions import desc

from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.types import IntegerType

## Obtain last_idx_peterman_line

In [3]:
'''
use eandev; 
drop table if exists eandev.yixli_last_idx_peterman_line;
create table eandev.yixli_last_idx_peterman_line as    
select  
  a.message_id,
  IF(b.message_id is null, 0, b.last_idx_peterman_line) as last_idx_peterman_line,
  IF(c.message_id is null, 0, c.last_hotel_result_index) as last_hotel_result_index
from eandev.HotelMessages as a 
left join (
  select  
   message_id
  ,MAX(v1.hotel_result_index) as last_idx_peterman_line
  from eandev.HotelMessages 
  lateral view explode(hotels) hotel_tbl as v1
  where load_date between '20191201' and '20200101'
  and ((v1.property_level_neg = true) or (v1.chain_level_neg = true) or (v1.preferred = true))
  and message_id is not null 
  and message_id <> ''
  and ((original_message_id is null) or (original_message_id == '')) 
  and type = 'HSR'  
  group by message_id
) as b ON a.message_id = b.message_id
left join (
  select  
   message_id
  ,MAX(v1.hotel_result_index) as last_hotel_result_index
  from eandev.HotelMessages 
  lateral view explode(hotels) hotel_tbl as v1
  where load_date between '20191201' and '20200101'
  and message_id is not null 
  and message_id <> ''
  and ((original_message_id is null) or (original_message_id == '')) 
  and type = 'HSR'  
  group by message_id
) as c ON a.message_id = c.message_id
where a.load_date between '20191201' and '20200101'
and a.message_id is not null 
and a.message_id <> ''
and ((original_message_id is null) or (original_message_id == '')) 
and a.type = 'HSR'
'''

"\nuse eandev; \ndrop table if exists eandev.yixli_last_idx_peterman_line;\ncreate table eandev.yixli_last_idx_peterman_line as    \nselect  \n  a.message_id,\n  IF(b.message_id is null, 0, b.last_idx_peterman_line) as last_idx_peterman_line,\n  IF(c.message_id is null, 0, c.last_hotel_result_index) as last_hotel_result_index\nfrom eandev.HotelMessages as a \nleft join (\n  select  \n   message_id\n  ,MAX(v1.hotel_result_index) as last_idx_peterman_line\n  from eandev.HotelMessages \n  lateral view explode(hotels) hotel_tbl as v1\n  where load_date between '20191201' and '20200101'\n  and ((v1.property_level_neg = true) or (v1.chain_level_neg = true) or (v1.preferred = true))\n  and message_id is not null \n  and message_id <> ''\n  and ((original_message_id is null) or (original_message_id == '')) \n  and type = 'HSR'  \n  group by message_id\n) as b ON a.message_id = b.message_id\nleft join (\n  select  \n   message_id\n  ,MAX(v1.hotel_result_index) as last_hotel_result_index\n  from

In [3]:
peterman_schema = T.StructType([
    T.StructField("message_id", T.StringType(), True),                  #1
    T.StructField("last_idx_peterman_line", T.IntegerType(), True),     #2 
    T.StructField("last_hotel_result_index", T.IntegerType(), True)     #3
])

In [4]:
file_loc = "s3://ege-ds-workshops-corp/yixli/last_idx_peterman_line.csv"

peterman_df = sqlContext.read.format('csv').\
            options(header='false', inferSchema='false', delimiter=',').\
            schema(peterman_schema).load(file_loc)

# load_date between '20191201' and '20200101'

print(peterman_df.count())

2252960

In [5]:
peterman_df.show(50)

+--------------------+----------------------+-----------------------+
|          message_id|last_idx_peterman_line|last_hotel_result_index|
+--------------------+----------------------+-----------------------+
|0010900b-b2e3-4e3...|                     0|                     29|
|003dfe2c-fa7b-4b7...|                     0|                      8|
|004c986f-7f97-4a4...|                     7|                     29|
|006abdad-d5ba-47d...|                     0|                      0|
|0081f9d5-9bf1-406...|                     0|                     18|
|009cca8a-5842-43c...|                     0|                      0|
|00dbaaf3-3bac-43c...|                     0|                     29|
|012f23f3-6cfb-40f...|                     0|                     29|
|01379dd8-37da-466...|                     0|                     29|
|0190fc4e-9dd2-4f8...|                     1|                     21|
|01924dd0-9d13-4e5...|                    11|                     16|
|01acce93-2407-4bf..

## Search data

In [ ]:
'''
use eandev;

drop table if exists eandev.yixli_search_commission;
create table eandev.yixli_search_commission as   
select 
message_id
,message_timestamp as message_date
,check_in_date
,check_out_date
,tuid
,v1.hotel_id
,v1.hotel_result_index
,v2.rate_index
,v2.price.amount as rate_amount
,v2.price.currency as rate_currency
,(case when v2.rate_type = 'GDS_CHAIN_NEGOTIATED' then 'NEG'                             
  when v2.rate_type = 'GDS_CONSORTIA' then 'GDS'
  when v2.rate_type = 'GDS_NEGOTIATED' then 'NEG'
  when v2.rate_type = 'GDS_PUBLISHED' then 'GDS'                             
  when v2.rate_type = 'EPR_MERCHANT' then 'EPRM'
  when v2.rate_type = 'ESR_DIRECT_AGENCY' then 'ESRA'
  when v2.rate_type = 'EPR_DIRECT_AGENCY' then 'EPRA'
  when v2.rate_type = 'ESR_MERCHANT' then 'ESRM'
  else v2.rate_type end) as rate_type
,NVL(v3.commission_base, 0.0) as commission_base
,NVL(v3.commission_tax, 0.0) as commission_tax
,NVL(v3.currency, 'USD') as currency
from eandev.HotelMessages
lateral view explode(travelers) travl_tbl as tuid
lateral view explode(hotels) hotel_tbl as v1
lateral view explode(v1.prices) rate_tbl as v2
lateral view explode(v2.commission_list) cms_tbl as v3 
where load_date between '20191201' and '20200101'
and message_id is not null 
and message_id <> ''
and v2.selected = True
and type = 'HSR'
'''

In [ ]:
'''
ssh Chwxedwhdc002.datawarehouse.expecn.com

hive -e "use eandev; INSERT OVERWRITE LOCAL DIRECTORY '/home/yixli/temp' ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' select * from yixli_search_commission;"&

cd temp
ls | xargs -I '{}' -n1 sed -i 's/\\N//g' '{}' &
cd ..
cat temp/* >search_data.csv


~/.linuxbrew/bin/aws_key_gen login

aws s3 cp search_data.csv s3://ege-ds-workshops-corp/yixli/search_data.csv


'''

In [5]:
search_schema = T.StructType([
    T.StructField("message_id", T.StringType(), True),                  #1
    T.StructField("message_date", T.TimestampType(), True),             #2
    T.StructField("check_in_date", T.DateType(), True),                 #3
    T.StructField("check_out_date", T.DateType(), True),                #4
    T.StructField("tuid", T.IntegerType(), True),                       #5
    T.StructField("hotel_id", T.IntegerType(), True),                   #6 
    T.StructField("hotel_result_index", T.IntegerType(), True),         #7
    T.StructField("rate_index", T.IntegerType(), True),                 #8 
    T.StructField("rate_amount", T.FloatType(), True),                  #9
    T.StructField("rate_currency", T.StringType(), True),               #10
    T.StructField("rate_type", T.StringType(), True),                   #11
])

In [6]:
file_loc = "s3://ege-ds-workshops-corp/yixli/search_data.csv"
print("Collecting search the data...")
search_df1 = sqlContext.read.format('csv').\
            options(header='false', inferSchema='false', delimiter=',').\
            schema(search_schema).load(file_loc)

# load_date between '20191201' and '20200101'

search_df1 = search_df1.\
            filter(F.col("hotel_id")>0).\
            filter(F.col("rate_amount").isNotNull())
print(search_df1.count())

278452443

In [7]:
search_df1.groupBy('rate_type').agg(F.count(F.col('message_id'))).show()

+---------+-----------------+
|rate_type|count(message_id)|
+---------+-----------------+
|     EPRA|         15888335|
|      NEG|          7205695|
|     EPRM|         31400764|
|     ESRM|        126081114|
|     ESRA|         57809894|
|      GDS|         40066641|
+---------+-----------------+

In [7]:
search_schema = T.StructType([
    T.StructField("message_id", T.StringType(), True),                  #1
    T.StructField("message_date", T.TimestampType(), True),             #2
    T.StructField("check_in_date", T.DateType(), True),                 #3
    T.StructField("check_out_date", T.DateType(), True),                #4
    T.StructField("tuid", T.IntegerType(), True),                       #5
    T.StructField("hotel_id", T.IntegerType(), True),                   #6 
    T.StructField("hotel_result_index", T.IntegerType(), True),         #7
    T.StructField("rate_index", T.IntegerType(), True),                 #8 
    T.StructField("rate_amount", T.FloatType(), True),                  #9
    T.StructField("rate_currency", T.StringType(), True),               #10
    T.StructField("rate_type", T.StringType(), True),                   #11
    T.StructField("commission_base", T.FloatType(), True),              #12
    T.StructField("base_category", T.StringType(), True),               #13
    T.StructField("commission_tax", T.FloatType(), True),               #14
    T.StructField("tax_category", T.StringType(), True),                #15
    T.StructField("commission_currency", T.StringType(), True)          #16
])


In [8]:
file_loc = "s3://ege-ds-workshops-corp/yixli/search_commission_month.csv"
print("Collecting search the data...")
search_df2 = sqlContext.read.format('csv').\
            options(header='false', inferSchema='false', delimiter=',').\
            schema(search_schema).load(file_loc)

# load_date between '20191201' and '20200101'

search_df2 = search_df2.\
            filter(F.col("hotel_id")>0).\
            filter(F.col("rate_amount").isNotNull())
print(search_df2.count())

222922918

In [10]:
search_df2.groupBy('rate_type').agg(F.count(F.col('message_id'))).show()

+---------+-----------------+
|rate_type|count(message_id)|
+---------+-----------------+
|     EPRA|         14898092|
|     EPRM|         28523882|
|     ESRM|        121541868|
|     ESRA|         57959076|
+---------+-----------------+

In [9]:
search_df = search_df1.\
             join(search_df2.select("commission_base","commission_tax","commission_currency","message_id","hotel_id","check_in_date", "check_out_date","tuid","rate_index",'rate_type'),
                  ["message_id","hotel_id","check_in_date", "check_out_date","tuid","rate_index",'rate_type']
                 ,how='left').\
             dropDuplicates()

In [11]:
search_df.groupBy('rate_type').agg(F.count(F.col('message_id'))).show()

+---------+-----------------+
|rate_type|count(message_id)|
+---------+-----------------+
|     EPRA|         17350683|
|      NEG|          7205666|
|     EPRM|         36257759|
|     ESRM|        145804059|
|     ESRA|         60765380|
|      GDS|         40066599|
+---------+-----------------+

In [10]:
search_df = search_df.withColumn("hotel_index", row_number().over(Window.partitionBy("message_id", "tuid").\
                                                        orderBy(F.asc("hotel_result_index"))))

In [13]:
search_df.printSchema()

root
 |-- message_id: string (nullable = true)
 |-- hotel_id: integer (nullable = true)
 |-- check_in_date: date (nullable = true)
 |-- check_out_date: date (nullable = true)
 |-- tuid: integer (nullable = true)
 |-- rate_index: integer (nullable = true)
 |-- rate_type: string (nullable = true)
 |-- message_date: timestamp (nullable = true)
 |-- hotel_result_index: integer (nullable = true)
 |-- rate_amount: float (nullable = true)
 |-- rate_currency: string (nullable = true)
 |-- commission_base: float (nullable = true)
 |-- commission_tax: float (nullable = true)
 |-- commission_currency: string (nullable = true)
 |-- hotel_index: integer (nullable = true)

In [11]:
dir = 's3://ege-ds-workshops-corp/yixli/data_understanding/'

datestamp = datetime.datetime.now().strftime('%m-%d-%Y')
search_df.repartition(1).write.format('com.databricks.spark.csv').mode('overwrite')\
.save(dir+datestamp+'_search_df.csv',header = 'true')

## Read search data

In [3]:
search_schema = T.StructType([
    T.StructField("message_id", T.StringType(), True),                  #1
    T.StructField("hotel_id", T.IntegerType(), True),                   #6
    T.StructField("check_in_date", T.DateType(), True),                 #3
    T.StructField("check_out_date", T.DateType(), True),                #4
    T.StructField("tuid", T.IntegerType(), True),                       #5
    T.StructField("rate_index", T.IntegerType(), True),                 #8 
    T.StructField("rate_type", T.StringType(), True),                   #11
    T.StructField("message_date", T.TimestampType(), True),             #2
    T.StructField("hotel_result_index", T.IntegerType(), True),         #7
    T.StructField("rate_amount", T.FloatType(), True),                  #9
    T.StructField("rate_currency", T.StringType(), True),               #10
    T.StructField("commission_base", T.FloatType(), True),              #12
    T.StructField("commission_tax", T.FloatType(), True),               #14
    T.StructField("commission_currency", T.StringType(), True),          #16
    T.StructField("hotel_index", T.IntegerType(), True)                 #8 
])


In [4]:
file_loc = "s3://ege-ds-workshops-corp/yixli/data_understanding/08-10-2020_search_df.csv"
print("Collecting search the data...")
search_df = sqlContext.read.format('csv').\
            options(header='false', inferSchema='false', delimiter=',').\
            schema(search_schema).load(file_loc)

# load_date between '20191201' and '20200101'
print(search_df.count())

307450147

In [5]:
search_df = search_df.\
            filter(F.col("hotel_id")>0).\
            filter(F.col("rate_amount").isNotNull())

In [19]:
search_df.groupBy('rate_type').agg(F.count(F.col('message_id'))).show()

+---------+-----------------+
|rate_type|count(message_id)|
+---------+-----------------+
|     EPRA|         17350683|
|      NEG|          7205666|
|     EPRM|         36257759|
|     ESRM|        145804059|
|     ESRA|         60765380|
|      GDS|         40066599|
+---------+-----------------+

## Filter hotels below peterman_line

In [17]:
search_df.registerTempTable("search_df")
peterman_df.registerTempTable("peterman_df")

In [18]:
search_below_peterman_df = sqlContext.sql("select\
                             b.*\
                             from search_df as b join peterman_df as c on (b.message_id=c.message_id)\
                             where (b.hotel_result_index>c.last_idx_peterman_line)\
                             order by b.message_id, b.hotel_result_index").\
                             dropDuplicates()

In [22]:
search_below_peterman_df.filter(F.col('rate_type')=='NEG').count()

758

In [19]:
search_below_peterman_df = search_below_peterman_df.filter(F.col('rate_type')!='NEG')

In [20]:
search_below_peterman_df.groupBy('rate_type').agg(F.count(F.col('message_id'))).show()

+---------+-----------------+
|rate_type|count(message_id)|
+---------+-----------------+
|     EPRA|         11914786|
|     EPRM|         25588796|
|     ESRM|        105851405|
|     ESRA|         44691781|
|      GDS|         23491335|
+---------+-----------------+

## Exchange currency to USD

In [21]:
sqlCurrency = """select DISTINCT
                 a.exchangeratedate as timestamp,
                 trunc(a.exchangeratedate) as exch_rate_date,
                 a.fromcurrencycode,
                 a.exchangerate
                 from public.exchangeratedailyfull a
                 where trunc(a.exchangeratedate)>=to_date('20191201','YYYYMMDD') 
                 and trunc(a.exchangeratedate)<=to_date('20200101','YYYYMMDD') 
                 and tocurrencycode = 'USD' """
connection_string = "jdbc:redshift://egencia-reporting.czjkedodj6lc.us-west-2.redshift.amazonaws.com:5439/egedatamart?user=ds_rpt_user&password=Jan2019!";

currency_df = sqlContext.read.\
    format("com.databricks.spark.redshift").\
    option("url", connection_string).\
    option("query", sqlCurrency).\
    option("tempdir", "s3a://ege-ds-workshops-corp/yixli/").\
    load().\
    dropDuplicates().\
    orderBy(["exch_rate_date","fromcurrencycode"],ascending=True)

In [22]:
local_below_peterman_df = search_below_peterman_df.filter(F.col('rate_type')!='GDS').filter(F.col('commission_base').isNotNull())
local_below_peterman_df.registerTempTable("local_below_peterman_df")
currency_df.registerTempTable("currency_df")

In [23]:
local_below_peterman_usd_df = sqlContext.sql("select\
                             a.*,\
                             a.rate_amount*COALESCE(ex1.exchangerate,1) AS src_rate_amount_usd,\
                             a.commission_base*COALESCE(ex2.exchangerate,1) AS src_commission_base_usd,\
                             a.commission_tax*COALESCE(ex2.exchangerate,1) AS src_commission_tax_usd\
                             from local_below_peterman_df a\
                             left join currency_df ex1 on to_date(a.message_date)=ex1.exch_rate_date and a.rate_currency=ex1.FromCurrencyCode\
                             left join currency_df ex2 on to_date(a.message_date)=ex2.exch_rate_date and a.commission_currency=ex2.FromCurrencyCode").\
                             dropDuplicates()

In [24]:
local_below_peterman_usd_df = local_below_peterman_usd_df.\
withColumn('src_supply_revenue_usd',F.col('src_commission_base_usd')+F.col('src_commission_tax_usd'))

In [25]:
local_below_peterman_usd_df = local_below_peterman_usd_df.filter(F.col('src_supply_revenue_usd')<1000)

In [26]:
local_below_peterman_usd_df.groupBy('rate_type').agg(F.count(F.col('message_id'))).show()

+---------+-----------------+
|rate_type|count(message_id)|
+---------+-----------------+
|     EPRA|          2831144|
|     EPRM|         10851577|
|     ESRM|         45737980|
|     ESRA|          7473111|
+---------+-----------------+

In [39]:
sqlGDS =  """SELECT 
      a.hotelid as hotel_id,
     trunc(a.issuedate) as issue_date,
     trunc(a.traveldatestart) as check_in_date,
     trunc(a.traveldateend) as check_out_date,
      (case when a.customersystemid=1 then a.TUIDTraveler else g.TUIDInternal end) as tuid,      
      a.travelproductid AS travel_product_id,
      a.GroupAccountID AS group_account_id,
      hsp.rate_type as hotel_rate_type,
      (case  
        when a.hotelrateTypeSupplyid IN (1,2,3,4,5,10,11,14,16,17,18,20,25,26,28,29,30,32) then 'GDS'
        when a.hotelrateTypeSupplyid IN (12,13,15,19,21,22,23,27,31,33,34) then 'NEG'
        when a.hotelrateTypeSupplyid = 6 then 'ESRA'
        when a.hotelrateTypeSupplyid = 7 then 'ESRM'
        when a.hotelrateTypeSupplyid = 8 then 'EPRA'
        when a.hotelrateTypeSupplyid = 9 then 'EPRM'
        when a.hotelrateTypeSupplyid IN (24,35) then 'HOTMIP'
        else 'UNK'
        end) as bk_rate_type,
      NVL(hsp.commission,0.00) As commission_rate,
      NVL(hsp.gds,0.00) AS gds
FROM public.HotelFact a
  JOIN TravelerAccountDim g 
    ON a.TUIDTraveler = g.TUID AND a.customersystemid=g.customersystemid 
  JOIN public.GroupAccountdim d 
    ON a.GroupAccountID = d.GroupAccountID 
  JOIN public.hotel_supply hsp 
    ON hsp.year = EXTRACT (year FROM a.IssueDate)
    AND hsp.quarter = EXTRACT (quarter FROM a.IssueDate)
    AND hsp.rate_type = CASE
         WHEN a.HotelRateTypeSupplyID IN (7,9,24) THEN 'Expedia Collect'
         WHEN a.HotelRateTypeSupplyID IN (6,8,35) THEN 'Hotel Collect'
         WHEN a.HotelRateTypeSupplyID IN (12,13,15,19,21,22,23,27,31,33,34) THEN 'Negotiated'
         ELSE 'Published & GDS' END
    AND hsp.travel_product_id = a.TravelProductID
WHERE a.CustomerSystemID IN (1,2)
AND   (d.groupaccountinternaltypeid = 1 OR d.groupaccountinternaltypeid = 3)
AND a.BookingTypeID in (1,3) 
and hsp.year=2019
and hsp.quarter=4""".replace('\n',' ')
            
connection_string = "jdbc:redshift://egencia-reporting.czjkedodj6lc.us-west-2.redshift.amazonaws.com:5439/egedatamart?user=ds_rpt_user&password=Jan2019!";

gds_df = sqlContext.read.\
    format("com.databricks.spark.redshift").\
    option("url", connection_string).\
    option("query", sqlGDS).\
    option("tempdir", "s3a://ege-ds-workshops-corp/yixli/").\
    load().\
    filter(F.col("hotel_rate_type")=='Published & GDS').\
    filter(F.col('commission_rate').isNotNull()).\
    filter(F.col('gds').isNotNull()).\
    dropDuplicates()

In [40]:
gds_df = gds_df.groupBy('hotel_id').agg(F.avg('commission_rate').alias('commission_rate'),
                                       F.avg('gds').alias('gds'))

In [41]:
gds_df.show(50)

+--------+--------------------+-----+
|hotel_id|     commission_rate|  gds|
+--------+--------------------+-----+
|   28024|0.080000000000000...|0E-22|
| 1805074|0.080000000000000...|0E-22|
|  525571|0.080000000000000...|0E-22|
|  578559|0.080000000000000...|0E-22|
|19681545|0.080000000000000...|0E-22|
|  807527|0.080000000000000...|0E-22|
|  528692|0.080000000000000...|0E-22|
|    5300|0.080000000000000...|0E-22|
|  904712|0.080000000000000...|0E-22|
|    7833|0.080000000000000...|0E-22|
|  802387|0.080000000000000...|0E-22|
| 5058675|0.080000000000000...|0E-22|
|  892857|0.080000000000000...|0E-22|
|23192964|0.080000000000000...|0E-22|
|11135961|0.080000000000000...|0E-22|
|18411003|0.080000000000000...|0E-22|
|    1829|0.080000000000000...|0E-22|
| 1144437|0.080000000000000...|0E-22|
|33374019|0.080000000000000...|0E-22|
|11785371|0.080000000000000...|0E-22|
|18629649|0.080000000000000...|0E-22|
| 2863595|0.080000000000000...|0E-22|
|    4818|0.080000000000000...|0E-22|
|    2366|0.

In [42]:
gds_below_peterman_df = search_below_peterman_df.filter(F.col('rate_type')=='GDS')
gds_below_peterman_df = gds_below_peterman_df.join(currency_df,
                                                                [F.to_date(gds_below_peterman_df.message_date)==currency_df.exch_rate_date,
                                                                gds_below_peterman_df.rate_currency==currency_df.fromcurrencycode]).\
                        withColumn('src_rate_amount_usd',gds_below_peterman_df.rate_amount*currency_df.exchangerate)                                             
gds_below_peterman_df.count()

23490685

In [43]:
gds_below_peterman_df = gds_below_peterman_df.\
join(gds_df.select('hotel_id','gds','commission_rate'),['hotel_id'],how='left').\
withColumn('src_supply_revenue_usd',F.col('src_rate_amount_usd')*F.col('commission_rate')+F.col('gds'))

In [44]:
gds_below_peterman_df = gds_below_peterman_df.filter(F.col('src_supply_revenue_usd')<1000)

In [45]:
gds_below_peterman_df.count()

19233691

In [57]:
gds_below_peterman_df = gds_below_peterman_df.withColumn('src_commission_base_usd',F.lit(0.0))

In [58]:
search_revenue_below_peterman_usd_df = local_below_peterman_usd_df.select('hotel_id','message_id','message_date',
                             'check_in_date','check_out_date','tuid',
                             'rate_index','rate_type','hotel_result_index','hotel_index',
                            'src_rate_amount_usd','src_commission_base_usd','src_supply_revenue_usd').\
                            union(gds_below_peterman_df.select('hotel_id','message_id','message_date',
                             'check_in_date','check_out_date','tuid',
                             'rate_index','rate_type','hotel_result_index','hotel_index',
                            'src_rate_amount_usd','src_commission_base_usd','src_supply_revenue_usd'))

In [47]:
search_revenue_below_peterman_usd_df.count()

86127503

In [ ]:
search_revenue_below_peterman_usd_df.groupBy('rate_type').agg(F.avg('src_supply_revenue_usd')).show()

In [60]:
dir = 's3://ege-ds-workshops-corp/yixli/data_understanding/'

datestamp = datetime.datetime.now().strftime('%m-%d-%Y')
search_revenue_below_peterman_usd_df.repartition(1).write.format('com.databricks.spark.csv').mode('overwrite')\
.save(dir+datestamp+'_search_revenue_below_peterman_usd_df.csv',header = 'true')

In [59]:
search_revenue_below_peterman_usd_df.printSchema()

root
 |-- hotel_id: integer (nullable = true)
 |-- message_id: string (nullable = true)
 |-- message_date: timestamp (nullable = true)
 |-- check_in_date: date (nullable = true)
 |-- check_out_date: date (nullable = true)
 |-- tuid: integer (nullable = true)
 |-- rate_index: integer (nullable = true)
 |-- rate_type: string (nullable = true)
 |-- hotel_result_index: integer (nullable = true)
 |-- hotel_index: integer (nullable = true)
 |-- src_rate_amount_usd: double (nullable = true)
 |-- src_commission_base_usd: double (nullable = true)
 |-- src_supply_revenue_usd: double (nullable = true)

In [6]:
search_schema = T.StructType([
    T.StructField("hotel_id", T.IntegerType(), True),                   #6
     T.StructField("message_id", T.StringType(), True),                  #1
     T.StructField("message_date", T.TimestampType(), True),             #2
    T.StructField("check_in_date", T.DateType(), True),                 #3
    T.StructField("check_out_date", T.DateType(), True),                #4
    T.StructField("tuid", T.IntegerType(), True),                       #5
    T.StructField("rate_index", T.IntegerType(), True),                 #8 
    T.StructField("rate_type", T.StringType(), True),                   #11
    T.StructField("hotel_result_index", T.IntegerType(), True),         #7
    T.StructField("hotel_index", T.IntegerType(), True),                 #8
    T.StructField("src_rate_amount_usd", T.FloatType(), True),              #12
    T.StructField("src_commission_base_usd", T.FloatType(), True),              #12
    T.StructField("src_supply_revenue_usd", T.FloatType(), True)              #14
])

In [7]:
file_loc = "s3://ege-ds-workshops-corp/yixli/data_understanding/08-11-2020_search_revenue_below_peterman_usd_df.csv"
print("Collecting search the data...")
search_revenue_below_peterman_usd_df = sqlContext.read.format('csv').\
            options(header='True', inferSchema='false', delimiter=',').\
            schema(search_schema).load(file_loc)

# load_date between '20191201' and '20200101'
print(search_revenue_below_peterman_usd_df.count())

86127503

In [52]:
search_revenue_below_peterman_usd_df.groupBy('rate_type').agg(F.count(F.col('message_id'))).show()

+---------+-----------------+
|rate_type|count(message_id)|
+---------+-----------------+
|     EPRA|          2831144|
|     EPRM|         10851577|
|     ESRM|         45737980|
|     ESRA|          7473111|
|      GDS|         19233691|
+---------+-----------------+

In [63]:
search_revenue_below_peterman_usd_df.\
                  groupBy('rate_type').\
                  agg(F.avg("src_rate_amount_usd"),F.avg("src_commission_base_usd"),F.avg("src_supply_revenue_usd")).show()

+---------+------------------------+----------------------------+---------------------------+
|rate_type|avg(src_rate_amount_usd)|avg(src_commission_base_usd)|avg(src_supply_revenue_usd)|
+---------+------------------------+----------------------------+---------------------------+
|     EPRA|      160.50747284356697|           25.78638466699946|         28.737860917031753|
|     EPRM|      182.55486579183457|          29.988307561942353|         38.195347010062655|
|     ESRM|      177.43720665719636|           27.06187241759471|          34.92993847592493|
|     ESRA|      150.86520571461784|           23.84484317570675|         26.644187551017772|
|      GDS|      191.35448903623652|                         0.0|         15.308359133772106|
+---------+------------------------+----------------------------+---------------------------+

In [37]:
%spark -o search_below_peterman_usd_df -n 30000

## Booking data

In [ ]:
'''
wget https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/1.2.20.1043/RedshiftJDBC42-no-awssdk-1.2.20.1043.jar
wget https://repo1.maven.org/maven2/com/databricks/spark-avro_2.11/3.2.0/spark-avro_2.11-3.2.0.jar
wget https://repo1.maven.org/maven2/com/databricks/spark-redshift_2.11/2.0.1/spark-redshift_2.11-2.0.1.jar
wget https://repo1.maven.org/maven2/com/databricks/spark-csv_2.11/1.5.0/spark-csv_2.11-1.5.0.jar
wget https://repo1.maven.org/maven2/com/eclipsesource/minimal-json/minimal-json/0.9.5/minimal-json-0.9.5.jar
sudo cp *.jar /usr/lib/spark/jars
'''

In [8]:
sqlBookings = """SELECT 
      a.hotelfactid,
      a.hotelid as hotel_id,
     trunc(a.issuedate) as issue_date,
     trunc(a.traveldatestart) as check_in_date,
     trunc(a.traveldateend) as check_out_date,
     hsp.quarter as quarter,
      (case when a.customersystemid=1 then a.TUIDTraveler else g.TUIDInternal end) as tuid,      
      a.travelproductid AS travel_product_id,
      a.GroupAccountID AS group_account_id,
      hsp.rate_type AS hotel_rate_type,
      (case  
        when a.hotelrateTypeSupplyid IN (1,2,3,4,5,10,11,14,16,17,18,20,25,26,28,29,30,32) then 'GDS'
        when a.hotelrateTypeSupplyid IN (12,13,15,19,21,22,23,27,31,33,34) then 'NEG'
        when a.hotelrateTypeSupplyid = 6 then 'ESRA'
        when a.hotelrateTypeSupplyid = 7 then 'ESRM'
        when a.hotelrateTypeSupplyid = 8 then 'EPRA'
        when a.hotelrateTypeSupplyid = 9 then 'EPRM'
        when a.hotelrateTypeSupplyid IN (24,35) then 'HOTMIP'
        else 'UNK'
        end) as bk_rate_type,
      CASE WHEN a.OnlineBool = 1 THEN 'online' ELSE 'offline' END AS booking_type,
      a.bookingamtgross::DOUBLE precision*COALESCE(ex.exchangerate,1::DOUBLE precision) AS gross_booking_value_usd,
      a.bookingamtcommissionest::DOUBLE precision*COALESCE(ex.exchangerate,1::DOUBLE precision)
        + a.bookingamtgross::DOUBLE precision*COALESCE(ex.exchangerate,1::DOUBLE precision)*NVL(hsp.commission,0.00) AS total_commission,
      a.bookingamtmargin::DOUBLE precision*COALESCE(ex.exchangerate,1::DOUBLE precision) AS total_markup,
      NVL(hsp.gds,0.00) AS total_gds,
      NVL(total_commission,0.00) + NVL(total_markup,0.00) + NVL(total_gds,0.00) AS total_supply_revenue
FROM public.HotelFact a
  JOIN public.ExchangeRateDailySubset ex
    ON a.IssueDateTimeID = ex.TimeID AND a.CurrencyCode = ex.FromCurrencyCode AND ex.ToCurrencyCode = 'USD'
  JOIN TravelerAccountDim g 
    ON a.TUIDTraveler = g.TUID AND a.customersystemid=g.customersystemid 
  JOIN public.GroupAccountdim d 
    ON a.GroupAccountID = d.GroupAccountID 
  JOIN public.hotel_supply hsp 
    ON hsp.year = EXTRACT (year FROM a.IssueDate)
    AND hsp.quarter = EXTRACT (quarter FROM a.IssueDate)
    AND hsp.rate_type = CASE
         WHEN a.HotelRateTypeSupplyID IN (7,9,24) THEN 'Expedia Collect'
         WHEN a.HotelRateTypeSupplyID IN (6,8,35) THEN 'Hotel Collect'
         WHEN a.HotelRateTypeSupplyID IN (12,13,15,19,21,22,23,27,31,33,34) THEN 'Negotiated'
         ELSE 'Published & GDS' END
    AND hsp.travel_product_id = a.TravelProductID
WHERE a.CustomerSystemID IN (1,2)
AND   (d.groupaccountinternaltypeid = 1 OR d.groupaccountinternaltypeid = 3)
AND a.BookingTypeID in (1,3) 
and a.issuedate>=to_date('20191201','YYYYMMDD') 
and a.issuedate<=to_date('20200101','YYYYMMDD')""".replace('\n',' ')

connection_string = "jdbc:redshift://egencia-reporting.czjkedodj6lc.us-west-2.redshift.amazonaws.com:5439/egedatamart?user=ds_rpt_user&password=Jan2019!";

bookings_df = sqlContext.read.\
    format("com.databricks.spark.redshift").\
    option("url", connection_string).\
    option("query", sqlBookings).\
    option("tempdir", "s3a://ege-ds-workshops-corp/yixli/").\
    load()

bookings_df.cache()
bookings_df.count()

460242

In [29]:
dir = 's3://ege-ds-workshops-corp/yixli/data_understanding/'
datestamp = datetime.datetime.now().strftime('%m-%d-%Y')
bookings_df.repartition(1).write.format('com.databricks.spark.csv').mode('overwrite')\
.save(dir+datestamp+'_bookings_df.csv',header = 'true')

In [18]:
bookings_df.\
                  groupBy('bk_rate_type').\
                  agg(F.avg("total_supply_revenue"),F.avg("total_commission"),F.avg("total_markup")).show(20)

+------------+-------------------------+---------------------+-----------------+
|bk_rate_type|avg(total_supply_revenue)|avg(total_commission)|avg(total_markup)|
+------------+-------------------------+---------------------+-----------------+
|        EPRA|         46.0896540718292|     46.0896540718292|              0.0|
|         NEG|                      0.0|                  0.0|              0.0|
|        EPRM|        55.29926517066873|                  0.0|55.29926517066873|
|        ESRM|        54.00079820764198|                  0.0|54.01410507738632|
|      HOTMIP|        58.34601692940934|  0.18765591199625353|58.15836101741308|
|        ESRA|        43.04767840746359|    43.04767840746359|              0.0|
|         GDS|        28.36807550209393|   29.486316720503424|              0.0|
+------------+-------------------------+---------------------+-----------------+

In [9]:
bookings_df = bookings_df.filter(F.col('bk_rate_type')!='NEG')
bookings_df.count()

340081

In [10]:
bookings_df = bookings_df.withColumn('rate_type',F.when(F.col('bk_rate_type')=='HOTMIP','ESRM').otherwise(F.col('bk_rate_type')))

In [52]:
bookings_df.groupBy('bk_rate_type').count().show()

+------------+------+
|bk_rate_type| count|
+------------+------+
|        EPRA| 14133|
|        EPRM| 45201|
|        ESRM|142069|
|      HOTMIP| 21660|
|        ESRA| 41233|
|         GDS| 75785|
+------------+------+

In [53]:
bookings_df.groupBy('rate_type').count().show()

+---------+------+
|rate_type| count|
+---------+------+
|     EPRA| 14133|
|     EPRM| 45201|
|     ESRM|163729|
|     ESRA| 41233|
|      GDS| 75785|
+---------+------+

In [19]:
bookings_df.groupBy('travel_product_id').count().orderBy(desc('count')).show()

+-----------------+------+
|travel_product_id| count|
+-----------------+------+
|            60000|128440|
|            60020| 70777|
|            60003| 31166|
|            60006| 26748|
|            60010| 16274|
|            60035|  8081|
|            60013|  6922|
|            60004|  6558|
|            61025|  6185|
|            60011|  5331|
|            60008|  4871|
|            60080|  4669|
|            60009|  4306|
|            61027|  3226|
|            60082|  2903|
|            60092|  2406|
|            60084|  1921|
|            60081|  1420|
|            60083|  1339|
|            60086|  1005|
+-----------------+------+
only showing top 20 rows

## Adjust the revenue by the length of stay

In [11]:
bookings_df = bookings_df.filter(F.col('total_supply_revenue')>0)

In [12]:
bookings_df = bookings_df.\
                     withColumn('duration', F.datediff(F.col("check_out_date"),F.col( "check_in_date")).cast(IntegerType()))

In [13]:
bookings_df = bookings_df.\
                     withColumn('avg_book_rate_amount_usd',F.col('gross_booking_value_usd')/F.col( "duration")).\
                     withColumn('bk_avg_commission_usd',F.col('total_commission')/F.col( "duration")).\
                     withColumn('bk_avg_markup_usd',F.col('total_markup')/F.col( "duration")).\
                     withColumn('bk_avg_revenue_usd',F.col('total_supply_revenue')/F.col( "duration"))

In [57]:
bookings_df.\
                  groupBy('bk_rate_type').\
                  agg(F.avg("bk_avg_revenue_usd"),F.avg("bk_avg_commission_usd"),F.avg("bk_avg_markup_usd")).show(20)

+------------+-----------------------+--------------------------+----------------------+
|bk_rate_type|avg(bk_avg_revenue_usd)|avg(bk_avg_commission_usd)|avg(bk_avg_markup_usd)|
+------------+-----------------------+--------------------------+----------------------+
|        EPRA|     22.770339986908912|        22.770339986908912|                   0.0|
|        EPRM|     27.509558886367884|                       0.0|    27.509558886367884|
|        ESRM|     25.099492101674105|                       0.0|    25.099492101674105|
|      HOTMIP|       28.1425649447915|       0.07082701419944325|    28.071737930592057|
|        ESRA|     20.160778320416547|        20.160778320416547|                   0.0|
|         GDS|     13.753334274583183|        13.753334274583183|                   0.0|
+------------+-----------------------+--------------------------+----------------------+

In [58]:
bookings_df.\
                  groupBy('rate_type').\
                  agg(F.avg("bk_avg_revenue_usd"),F.avg("bk_avg_commission_usd"),F.avg("bk_avg_markup_usd")).show(20)

+---------+-----------------------+--------------------------+----------------------+
|rate_type|avg(bk_avg_revenue_usd)|avg(bk_avg_commission_usd)|avg(bk_avg_markup_usd)|
+---------+-----------------------+--------------------------+----------------------+
|     EPRA|     22.770339986908915|        22.770339986908915|                   0.0|
|     EPRM|     27.509558886367877|                       0.0|    27.509558886367877|
|     ESRM|      25.50236527755179|      0.009376806149416746|    25.492988471402366|
|     ESRA|     20.160778320416547|        20.160778320416547|                   0.0|
|      GDS|     13.753334274583183|        13.753334274583183|                   0.0|
+---------+-----------------------+--------------------------+----------------------+

## Matching last message in search chain with booking record

In [59]:
booking_search_df_no_filter = bookings_df.\
                join(search_df,["hotel_id","check_in_date", "check_out_date","tuid",
                                'rate_type']).\
                withColumn("rn", row_number().over(Window.partitionBy("hotel_id","check_in_date","check_out_date","tuid").orderBy(desc("message_date")))).\
                filter(F.col("rn") == 1).\
                withColumnRenamed("hotel_index","bk_hotel_index")
booking_search_df_no_filter.count()

319706

In [14]:
booking_search_df = bookings_df.\
                join(search_revenue_below_peterman_usd_df,["hotel_id","check_in_date", "check_out_date","tuid",'rate_type']).\
                withColumn("rn", row_number().over(Window.partitionBy("hotel_id","check_in_date","check_out_date","tuid").orderBy(desc("message_date")))).\
                filter(F.col("rn") == 1).\
                withColumnRenamed("hotel_index","bk_hotel_index")
booking_search_df.count()

83577

In [61]:
booking_search_df.groupBy('travel_product_id').count().orderBy(desc('count')).show()

+-----------------+-----+
|travel_product_id|count|
+-----------------+-----+
|            60000|36440|
|            60020|16339|
|            60003| 7478|
|            60006| 6657|
|            60010| 3462|
|            60035| 1758|
|            60004| 1471|
|            60013| 1341|
|            60008| 1103|
|            60011| 1052|
|            61025| 1047|
|            60009|  972|
|            60080|  885|
|            61027|  659|
|            60092|  514|
|            60084|  488|
|            60082|  482|
|            60081|  324|
|            60086|  301|
|            60083|  296|
+-----------------+-----+
only showing top 20 rows

In [62]:
booking_search_df.groupBy('bk_rate_type').count().show()

+------------+-----+
|bk_rate_type|count|
+------------+-----+
|        EPRA| 1856|
|        EPRM| 9735|
|        ESRM|31557|
|      HOTMIP| 4955|
|        ESRA| 4378|
|         GDS|31096|
+------------+-----+

In [23]:
booking_search_df.printSchema()

root
 |-- hotel_id: integer (nullable = true)
 |-- check_in_date: date (nullable = true)
 |-- check_out_date: date (nullable = true)
 |-- tuid: integer (nullable = true)
 |-- hotelfactid: integer (nullable = true)
 |-- issue_date: date (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- travel_product_id: integer (nullable = true)
 |-- group_account_id: integer (nullable = true)
 |-- hotel_rate_type: string (nullable = true)
 |-- bk_rate_type: string (nullable = true)
 |-- booking_type: string (nullable = true)
 |-- gross_booking_value_usd: double (nullable = true)
 |-- total_commission: double (nullable = true)
 |-- total_markup: double (nullable = true)
 |-- total_gds: decimal(20,18) (nullable = true)
 |-- total_supply_revenue: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- avg_book_rate_amount_usd: double (nullable = true)
 |-- bk_avg_commission_usd: double (nullable = true)
 |-- bk_avg_markup_usd: double (nullable = true)
 |-- bk_avg_revenue_usd: 

## Match rates: is booked/not booked

In [15]:
booking_search_df = booking_search_df.\
                         withColumn('rate_diff',F.abs(F.col('avg_book_rate_amount_usd')-F.col('src_rate_amount_usd')))

In [16]:
booking_search_df = booking_search_df.\
                         withColumn('is_booked',F.when((F.col('rate_diff')<1.0) & (F.col('bk_rate_type')==F.col('rate_type')),1).otherwise(0))

In [17]:
booking_search_df.filter(F.col('is_booked')==1).count()

19580

In [66]:
booking_search_df = booking_search_df.\
                    withColumn('revenue_diff1',F.abs(F.col('bk_avg_revenue_usd')-F.col('src_commission_base_usd'))).\
                    withColumn('revenue_diff2',F.abs(F.col('bk_avg_revenue_usd')-F.col('src_supply_revenue_usd'))).\
                    withColumn('relative_revenue_diff1',F.abs(F.col('bk_avg_revenue_usd')-F.col('src_commission_base_usd'))/F.col('src_commission_base_usd')).\
                    withColumn('relative_revenue_diff2',F.abs(F.col('bk_avg_revenue_usd')-F.col('src_supply_revenue_usd'))/F.col('src_supply_revenue_usd'))

In [67]:
booking_search_df.printSchema()

root
 |-- hotel_id: integer (nullable = true)
 |-- check_in_date: date (nullable = true)
 |-- check_out_date: date (nullable = true)
 |-- tuid: integer (nullable = true)
 |-- rate_type: string (nullable = true)
 |-- hotelfactid: integer (nullable = true)
 |-- issue_date: date (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- travel_product_id: integer (nullable = true)
 |-- group_account_id: integer (nullable = true)
 |-- hotel_rate_type: string (nullable = true)
 |-- bk_rate_type: string (nullable = true)
 |-- booking_type: string (nullable = true)
 |-- gross_booking_value_usd: double (nullable = true)
 |-- total_commission: double (nullable = true)
 |-- total_markup: double (nullable = true)
 |-- total_gds: decimal(20,18) (nullable = true)
 |-- total_supply_revenue: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- avg_book_rate_amount_usd: double (nullable = true)
 |-- bk_avg_commission_usd: double (nullable = true)
 |-- bk_avg_markup_usd: double (n

In [72]:
%spark -o booking_search_df -n 10000

In [73]:
%local
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:.5f}'.format

In [82]:
%local
def revenue_diff_hist(df,revenue):
    df = df.sort_values(by=['bk_rate_type'])
    fig = plt.figure(figsize=(8,6))
    fig = sns.boxplot(x='rate_type', y=revenue,data=df)
    fig.set(xlabel='rate_type', ylabel='revenue difference')
    fig.set_ylim(bottom=0,top=13)
    plt.tight_layout()
    return fig

In [83]:
%local
fig1 = revenue_diff_hist(booking_search_df,'revenue_diff1')
fig1.figure.savefig('image/revenue_diff1.png')

fig2 = revenue_diff_hist(booking_search_df,'revenue_diff2')
fig2.figure.savefig('image/revenue_diff2.png')




In [68]:
booking_search_df.filter(F.col('relative_revenue_diff1')>0.15).groupBy('bk_rate_type').count().show()

+------------+-----+
|bk_rate_type|count|
+------------+-----+
|        EPRA|  550|
|        EPRM| 6722|
|        ESRM|23208|
|      HOTMIP| 3623|
|        ESRA| 1471|
+------------+-----+

In [69]:
booking_search_df.filter(F.col('relative_revenue_diff2')>0.1).groupBy('bk_rate_type').count().show()

+------------+-----+
|bk_rate_type|count|
+------------+-----+
|        EPRA|  678|
|        EPRM| 3683|
|        ESRM|14352|
|      HOTMIP| 3106|
|        ESRA| 1793|
|         GDS|18502|
+------------+-----+

In [85]:
booking_search_df.filter(F.col('revenue_diff2')>4).groupBy('rate_type').count().show()

+---------+-----+
|rate_type|count|
+---------+-----+
|     EPRA|  370|
|     EPRM| 2620|
|     ESRM|11852|
|     ESRA|  952|
|      GDS| 1795|
+---------+-----+

In [86]:
booking_search_df.filter(F.col('revenue_diff2')>2).groupBy('rate_type').count().show()

+---------+-----+
|rate_type|count|
+---------+-----+
|     EPRA|  683|
|     EPRM| 4224|
|     ESRM|18616|
|     ESRA| 1696|
|      GDS| 7820|
+---------+-----+

In [71]:
booking_search_df.select('bk_rate_type','travel_product_id','duration','bk_avg_revenue_usd','src_commission_base_usd').\
                  orderBy(desc('revenue_diff2')).show(50)

+------------+-----------------+--------+------------------+-----------------------+
|bk_rate_type|travel_product_id|duration|bk_avg_revenue_usd|src_commission_base_usd|
+------------+-----------------+--------+------------------+-----------------------+
|         GDS|            60000|       1|           42.0224|                    0.0|
|      HOTMIP|            60000|       3|            121.13|                  348.0|
|        ESRM|            60000|       3|            181.25|                 344.85|
|        ESRM|            60000|       3|59.626666666666665|                 274.67|
|        ESRM|            60000|       2|            163.61|                 286.86|
|        ESRM|            60000|       1|            212.54|                 347.08|
|        ESRM|            60000|       6| 80.75023333333333|              208.96822|
|        ESRA|            60003|       2|15.049504950495047|              164.85808|
|        ESRA|            60003|       2|15.049504950495047|     

In [48]:
dir = 's3://ege-ds-workshops-corp/yixli/data_understanding/'

In [49]:
datestamp = datetime.datetime.now().strftime('%m-%d-%Y')
booking_search_df.repartition(1).write.format('com.databricks.spark.csv').mode('overwrite')\
.save(dir+datestamp+'_booking_search_df.csv',header = 'true')

## Match labels back to search data

In [87]:
booking_search_df.printSchema()

root
 |-- hotel_id: integer (nullable = true)
 |-- check_in_date: date (nullable = true)
 |-- check_out_date: date (nullable = true)
 |-- tuid: integer (nullable = true)
 |-- rate_type: string (nullable = true)
 |-- hotelfactid: integer (nullable = true)
 |-- issue_date: date (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- travel_product_id: integer (nullable = true)
 |-- group_account_id: integer (nullable = true)
 |-- hotel_rate_type: string (nullable = true)
 |-- bk_rate_type: string (nullable = true)
 |-- booking_type: string (nullable = true)
 |-- gross_booking_value_usd: double (nullable = true)
 |-- total_commission: double (nullable = true)
 |-- total_markup: double (nullable = true)
 |-- total_gds: decimal(20,18) (nullable = true)
 |-- total_supply_revenue: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- avg_book_rate_amount_usd: double (nullable = true)
 |-- bk_avg_commission_usd: double (nullable = true)
 |-- bk_avg_markup_usd: double (n

In [18]:
search_revenue_below_peterman_usd_label_df = search_revenue_below_peterman_usd_df.\
                join(booking_search_df.select('is_booked',"message_id","hotel_id","check_in_date", "check_out_date","tuid",'rate_type','rate_index'),
                     ["message_id","hotel_id","check_in_date", "check_out_date","tuid",'rate_type','rate_index'],
                     how='left').\
                withColumn("label", F.when(F.col('is_booked')==1,1).otherwise(0))
search_revenue_below_peterman_usd_label_df.count()

86127503

In [19]:
search_revenue_below_peterman_usd_label_df.filter(F.col('label')==1).count()

23261

In [20]:
search_revenue_below_peterman_usd_label_df.filter(F.col('label')==0).count()

86104415

In [23]:
search_revenue_below_peterman_usd_label_df.printSchema()

root
 |-- message_id: string (nullable = true)
 |-- hotel_id: integer (nullable = true)
 |-- check_in_date: date (nullable = true)
 |-- check_out_date: date (nullable = true)
 |-- tuid: integer (nullable = true)
 |-- rate_type: string (nullable = true)
 |-- rate_index: integer (nullable = true)
 |-- message_date: timestamp (nullable = true)
 |-- hotel_result_index: integer (nullable = true)
 |-- hotel_index: integer (nullable = true)
 |-- src_rate_amount_usd: float (nullable = true)
 |-- src_commission_base_usd: float (nullable = true)
 |-- src_supply_revenue_usd: float (nullable = true)
 |-- is_booked: integer (nullable = true)
 |-- label: integer (nullable = false)

In [22]:
dir = 's3://ege-ds-workshops-corp/yixli/data_understanding/'
datestamp = datetime.datetime.now().strftime('%m-%d-%Y')
search_revenue_below_peterman_usd_label_df.repartition(1).write.format('com.databricks.spark.csv').mode('overwrite')\
.save(dir+datestamp+'_search_revenue_below_peterman_usd_label_dfa.csv',header = 'true')

## Estimated revenue from booking data

In [81]:
bookings_df.\
                  groupBy('rate_type').\
                  agg(F.avg("bk_avg_revenue_usd"),F.avg("bk_avg_commission_usd"),F.avg("bk_avg_markup_usd")).show(20)

+---------+-----------------------+--------------------------+----------------------+
|rate_type|avg(bk_avg_revenue_usd)|avg(bk_avg_commission_usd)|avg(bk_avg_markup_usd)|
+---------+-----------------------+--------------------------+----------------------+
|     EPRA|     22.770339986908915|        22.770339986908915|                   0.0|
|     EPRM|     27.509558886367877|                       0.0|    27.509558886367877|
|     ESRM|      25.50236527755179|      0.009376806149416746|    25.492988471402366|
|     ESRA|     20.160778320416547|        20.160778320416547|                   0.0|
|      GDS|     13.753334274583183|        13.753334274583183|                   0.0|
+---------+-----------------------+--------------------------+----------------------+

## Estimated revenue from Search data(all)

In [79]:
booking_search_df.\
                  groupBy('rate_type').\
                  agg(F.avg("src_commission_base_usd"),F.avg(F.col("src_supply_revenue_usd"))).show(20)

+---------+----------------------------+---------------------------+
|rate_type|avg(src_commission_base_usd)|avg(src_supply_revenue_usd)|
+---------+----------------------------+---------------------------+
|     EPRA|          21.687513998282636|          24.03620434476077|
|     EPRM|           23.73457794222754|         30.228037416879875|
|     ESRM|           21.49949521765525|          27.81141515292509|
|     ESRA|           19.35225252128575|         21.406924983947718|
|      GDS|                         0.0|         12.537897125821159|
+---------+----------------------------+---------------------------+

## Estimated revenue from Search data(is booked)

In [80]:
booking_search_df.filter(F.col('is_booked')==1).\
                  groupBy('rate_type').\
                  agg(F.avg("src_commission_base_usd"),F.avg(F.col("src_supply_revenue_usd"))).show(20)

+---------+----------------------------+---------------------------+
|rate_type|avg(src_commission_base_usd)|avg(src_supply_revenue_usd)|
+---------+----------------------------+---------------------------+
|     EPRA|           19.00152848300807|         21.049423043849462|
|     EPRM|          18.799430768026244|         23.272720480296346|
|     ESRM|          16.611169259773764|          21.12176178395332|
|     ESRA|           17.01638980266414|          18.75087200991715|
|      GDS|                         0.0|         10.605623845375613|
+---------+----------------------------+---------------------------+

## Probabilty of booking by rate types

In [78]:
booking_search_df.\
                  groupBy('bk_rate_type').agg(F.count(F.col('is_booked')), F.avg(F.col('is_booked'))).show(20)

+------------+----------------+-------------------+
|bk_rate_type|count(is_booked)|     avg(is_booked)|
+------------+----------------+-------------------+
|        EPRA|            1852| 0.4924406047516199|
|        EPRM|            9739|0.29387000718759626|
|        ESRM|           31560| 0.2282319391634981|
|      HOTMIP|            4949|                0.0|
|        ESRA|            4375|             0.4256|
|         GDS|           31102|0.21664201659057294|
+------------+----------------+-------------------+

## Probabilty of booking by rate index

In [46]:
booking_search_df.\
                  groupBy('rate_index').agg(F.count(F.col('is_booked')), F.avg(F.col('is_booked'))).show(20)

+----------+----------------+--------------------+
|rate_index|count(is_booked)|      avg(is_booked)|
+----------+----------------+--------------------+
|        12|             284| 0.02464788732394366|
|         1|           10330|  0.1841239109390126|
|        13|             179|  0.0111731843575419|
|         6|            3755| 0.07163781624500666|
|         3|            6075| 0.12609053497942388|
|         5|            4681| 0.07968382824182867|
|        15|               9|                 0.0|
|         9|            1392| 0.03735632183908046|
|         4|            5201| 0.09402038069601999|
|         8|            3201| 0.05529522024367385|
|         7|            3509| 0.06497577657452265|
|        10|            1080| 0.03333333333333333|
|        11|             939| 0.01597444089456869|
|        14|             162|0.012345679012345678|
|         2|            8246| 0.11872422992966286|
|         0|           13800| 0.27956521739130435|
+----------+----------------+--

In [94]:
booking_search_df.\
                  groupBy('travel_product_id').agg(F.count(F.col('is_booked')), F.avg(F.col('is_booked'))).orderBy(desc(F.count(F.col('is_booked')))).show(20)

An error occurred while calling z:org.apache.spark.sql.functions.desc. Trace:
py4j.Py4JException: Method desc([class org.apache.spark.sql.Column]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:339)
	at py4j.Gateway.invoke(Gateway.java:276)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)


Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 44, in _
    jc = getattr(sc._jvm.functions, name)(col._jc if isinstance(col, Column) else col)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zi